# Homework 4: Prompt-based NLP
### Seonho Woo (clairewo@umich.edu)

In [1]:
pip install transformers

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install accelerate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install openprompt

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install evaluate

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import numpy as np
import pandas as pd
import os 
from collections import Counter, defaultdict
#import datasets
#from datasets import Dataset, DatasetDict

In [7]:
pip install torch

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
import torch
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, precision_recall_fscore_support
from tqdm import tqdm
import evaluate
#from sklearn.dummy import DummyClassifier
#from datasets import load_dataset
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

2023-04-26 09:13:08.952148: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-26 09:13:10.222361: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-26 09:13:16.539357: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## **Prompt-based Learning**

## Problem 1. 
(10 points) Write a simple piece code that takes a single word as input and then
tokenizes it with the BERT tokenizer in huggingface and returns the word’s corresponding tokens (or token IDs) in the BERT vocabulary. You’ll want to use this piece of code in the next task to check that your verbalizer is using only single-token words.

In [9]:
# Tokenize train and evaluation datasets with BertTokenizer
from transformers import BertTokenizer
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace

In [13]:
entry = input("Enter a single word: ")

Enter a single word:  tokenizing


In [14]:
entry

'tokenizing'

In [ ]:
from transformers import BertTokenizer

def tokenize_word(word):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    tokenizer.pre_tokenizer = Whitespace()
    tokens = tokenizer.tokenize(word)
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    return tokens, token_ids

In [ ]:
tokens, token_ids = tokenize_word(entry)
print(tokens, token_ids)
#tokens = tokenizer.convert_ids_to_tokens(tokenized_sentence)

['token', '##izing'] [19204, 6026]


## **Few-shot Learning**

## Problem 2. 
(20 points) Write 10 different prompts that can be used to classify toxic speech. Prompts should be relatively different (not just adding/changing one word). For each, come up with at least 2 verbalizations of each class (toxic/non-toxic). You can share verbalizations across prompts if needed. We really want to see some creativity across your prompts (this will also help the model learn more too).

In [10]:
import pandas as pd
from sklearn.metrics import f1_score
from tqdm import tqdm
import torch
import numpy as np

In [11]:
from transformers import BertTokenizer, AutoModelForSequenceClassification
from torch.optim import AdamW

In [12]:
from openprompt.prompts import ManualTemplate
from openprompt.prompts import ManualVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader

In [13]:
plm_object = load_plm("bert", "microsoft/MiniLM-L12-H384-uncased")
tokenizer = plm_object[1]

Some weights of BertForMaskedLM were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
classes = ["toxic", "nontoxic"]

In [15]:
prompt_template_1 = ManualTemplate(
    text = '{"placeholder":"text_a"}. Were there swear words in the previous sentence? {"mask"}',
    tokenizer = tokenizer,
)

prompt_verbalizer_1 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["yes"],
        "nontoxic": ["no"],
    },
    tokenizer = tokenizer,
)

prompt_template_2 = ManualTemplate(
    text = '{"placeholder":"text_a"}. On a scale of 1-5, how rude was that sentence? {"mask"}',
    tokenizer = tokenizer,
)

prompt_verbalizer_2 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["4", "5"],
        "nontoxic": ["1", "2", "3"],
    },
    tokenizer = tokenizer,
)

prompt_template_3 = ManualTemplate(
    text = '{"placeholder":"text_a"}. What was the author of the previous sentence feeling? {"mask"}',
    tokenizer = tokenizer,
)

prompt_verbalizer_3 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["anger", "frustration", "sad", "depressed"],
        "nontoxic": ["content", "happy", "calm", "nothing"],
    },
    tokenizer = tokenizer,
)

prompt_template_4 = ManualTemplate(
    text = '{"placeholder":"text_a"}. Would you kiss your mother with that mouth? {"mask"}',
    tokenizer = tokenizer,
)

prompt_verbalizer_4 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["no"],
        "nontoxic": ["yes"],
    },
    tokenizer = tokenizer,
)

prompt_template_5 = ManualTemplate(
    text = '{"placeholder":"text_a"}. If that sentence were spoken out, how loud would it be on a scale of 1-5? {"mask"}',
    tokenizer = tokenizer,
)


prompt_verbalizer_5 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["4", "5"],
        "nontoxic": ["1", "2", "3"],
    },
    tokenizer = tokenizer,
)

prompt_template_6 = ManualTemplate(
    text = 'Alice: How are you? Bob: {"placeholder":"text_a"}. Alice: That\'s {"mask"}',
    tokenizer = tokenizer,
)


prompt_verbalizer_6 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["rude", "terrible", "bad"],
        "nontoxic": ["nice", "good", "alright"],
    },
    tokenizer = tokenizer,
)

prompt_template_7 = ManualTemplate(
    text = 'If someone said {"placeholder":"text_a"} that would be {"mask"}',
    tokenizer = tokenizer,
)

prompt_verbalizer_7 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["rude", "terrible", "bad"],
        "nontoxic": ["nice", "good", "alright"],
    },
    tokenizer = tokenizer,
)

prompt_template_8 = ManualTemplate(
    text = 'If I agree with {"placeholder":"text_a"} then I am {"mask"}',
    tokenizer = tokenizer,
)

prompt_verbalizer_8 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["rude", "terrible", "dick", "horrible"],
        "nontoxic": ["nice", "good", "alright", "calm", "composed"],
    },
    tokenizer = tokenizer,
)

prompt_template_9 = ManualTemplate(
    text = '''
    YOU GAY ASSHOLE MOTHERFUCKER, THAT'S BULLSHIT. Is this toxic? yes
    Well, that's alright, you didn't know. Is this toxic? no
    {"placeholder":"text_a"}. Is this toxic? {"mask"}
    ''',
    tokenizer = tokenizer,
)

prompt_verbalizer_9 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["yes"],
        "nontoxic": ["no"],
    },
    tokenizer = tokenizer,
)

prompt_template_10 = ManualTemplate(
    text = 'As a Reddit mod would you ban this comment: {"placeholder":"text_a"}? {"mask"}',
    tokenizer = tokenizer,
)

prompt_verbalizer_10 = ManualVerbalizer(
    classes = classes,
    label_words = {
        "toxic": ["yes"],
        "nontoxic": ["no"],
    },
    tokenizer = tokenizer,
)

In [ ]:
# prompt1 = "What do you think of people who engage in this behavior?"
# prompt2 = "What is your opinion on using derogatory terms?"
# prompt3 = "Do you believe in equal rights for all people, regardless of their identity?"
# prompt4 = "How would you react to someone with a different opinion than yours?"
# prompt5 = "What is your stance on cyberbullying?"
# prompt6 = "What are your thoughts on using violence to solve conflicts?"
# prompt7 = "How do you feel about people who are different from you?"
# prompt8 = "What is your opinion on making offensive jokes?"
# prompt9 = "What is your opinion on making offensive jokes?"
# prompt10 = "What is your opinion on making offensive jokes?"

In [ ]:
class PromptClassifier:
    def __init__(self, template, verbalizer, plm_obj):
        self.plm, self.tokenizer, self.model_config, self.WrapperClass = plm_obj
        self.template = template
        self.verbalizer = verbalizer
        self.model = PromptForClassification(
            template = template,
            plm = self.plm,
            verbalizer = verbalizer,
        )
        self.criterion = torch.nn.BCELoss()
        self.optim = AdamW(self.model.parameters(), lr=1e-2)
    
    def create_data_loader(self, dataset):
        return PromptDataLoader(
            dataset = dataset,
            tokenizer = self.tokenizer,
            template = self.template,
            tokenizer_wrapper_class = self.WrapperClass,
        )

    def train(self, dataset):
        for batch in tqdm(self.create_data_loader(dataset)):
            logits = self.model(batch)
            preds = torch.argmax(logits, dim = -1)
            loss = self.criterion(preds.float(), batch["label"].float())
            loss = torch.autograd.Variable(loss, requires_grad = True)
            self.optim.step()
            self.optim.zero_grad()
            loss.backward()

    def evaluate(self, eval_dataset):
        self.model.eval()
        y_pred = []
        y_true = []
        with torch.no_grad():
            for batch in tqdm(self.create_data_loader(eval_dataset)):
                logits = self.model(batch)
                preds = torch.argmax(logits, dim = -1)
                y_pred.append(preds.item())
                y_true.append(batch["label"].item())    
        return f1_score(y_true, y_pred)

In [16]:
train_df = pd.read_csv("jigsaw-train.csv")
eval_df = pd.read_csv("jigsaw-dev.csv")

In [17]:
fewshot_classifiers = []
for i in tqdm(range(10)):
    train_dataset_0 = [
        InputExample(
            guid=idx,
            text_a=row["comment_text"],
            label=row["toxic"]
        ) for idx, row in train_df[train_df["toxic"] == 0].sample(5).iterrows()
    ]

    train_dataset_1 = [
        InputExample(
            guid=idx,
            text_a=row["comment_text"],
            label=row["toxic"]
        ) for idx, row in train_df[train_df["toxic"] == 1].sample(5).iterrows()
    ]

    train_dataset = train_dataset_0 + train_dataset_1

    eval_dataset_0 = [
        InputExample(
            guid=idx,
            text_a=row["comment_text"],
            label=row["toxic"]
        ) for idx, row in eval_df[eval_df["toxic"] == 0].sample(5).iterrows()
    ]

    eval_dataset_1 = [
        InputExample(
            guid=idx,
            text_a=row["comment_text"],
            label=row["toxic"]
        ) for idx, row in eval_df[eval_df["toxic"] == 1].sample(5).iterrows()
    ]

    eval_dataset = eval_dataset_0 + eval_dataset_1
    
    fsclassifiers = []
    fs_f1_scores = []
    for i in range(1, 11):
        prompt_classifier = PromptClassifier(eval("prompt_template_" + str(i)), eval("prompt_verbalizer_" + str(i)), plm_object)
        prompt_classifier.train(train_dataset)
        fsclassifiers.append(prompt_classifier)
        score = prompt_classifier.evaluate(eval_dataset)
        fs_f1_scores.append(score)
    fewshot_classifiers.append(np.argmax(fs_f1_scores))
    
fewshot_model_idx = max(set(fewshot_classifiers), key=fewshot_classifiers.count)
fewshot_model = fsclassifiers[fewshot_model_idx]

  0%|          | 0/10 [00:00<?, ?it/s]


NameError: name 'PromptClassifier' is not defined

In [ ]:
fewshot_model_idx

7

In [ ]:
fewshot_model_idx, fs_f1_scores

(7,
 [0.5,
  0.0,
  0.6666666666666666,
  0.6666666666666666,
  0.0,
  0.0,
  0.0,
  0.6666666666666666,
  0.4000000000000001,
  0.5714285714285714])

## **Zero-shot Learning**

## Problem 3. 
(20 points) Write 10 different zero-shot prompts that can be used to classify toxic speech. Prompts should be relatively different (not just adding/changing one word). We want you to try exploring the space of how to create a prompt. Some of the questions above can be used for inspiration, but there is a lot of room to explore here.

In [9]:
from sklearn.metrics import f1_score
from tqdm import tqdm

In [10]:
#pip install tensorflow

In [11]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding, AutoTokenizer, AutoModelForSequenceClassification 

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

In [13]:
class ZeroShotClassifier:
    def __init__(self, template, verbalizer, tokenizer, model):
        self.tokenizer = tokenizer
        self.template = template
        self.verbalizer = verbalizer
        self.model = model
        self.criterion = torch.nn.BCELoss()

    def evaluate(self, eval_dataset, eval=True):
        y_pred = []
        y_true = []
        with torch.no_grad():
            for idx, row in tqdm(eval_dataset.iterrows()):
                input = self.template.format(row["comment_text"])
                input = self.tokenizer(input, return_tensors="pt")
                output = self.model.generate(**input, temperature=0.8, max_new_tokens=500)
                output = tokenizer.batch_decode(output, skip_special_tokens=True)
                Flag = False
                for verb in self.verbalizer:
                    if not Flag:
                        for label in self.verbalizer[verb]:
                            if label in output[0].lower():
                                y_pred.append(verb)
                                Flag = True
                                break
                if not Flag:
                    y_pred.append("nontoxic") # guess most frequent class if no answer is available
                if eval:
                    y_true.append(row["toxic"])
        y_pred = list(map(lambda yi: 1 if yi == "toxic" else 0, y_pred))
        return [y_pred, f1_score(y_true, y_pred) if eval else None]

In [14]:
verbalizer1 = {"toxic": ["yes"], "nontoxic": ["no"]}
prompt1 = """
Text: I disagree with your opinion, but I respect your right to hold it.
Label: no
Text: You're an idiot for thinking that way.
Label: yes
Text: {}
Label:
"""

verbalizer2 = {"toxic": ["yes"], "nontoxic": ["no"]}
prompt2 = """
Below is an example of a instagram comment:

{}

If this comment is being rude or contains bullying words, reply yes. Otherwise, type no.
"""

verbalizer3 = {"toxic": ["A"], "nontoxic": ["B"]}
prompt3 = """
There are two people A and B who is each being toxic and nontoxic.

A. "I'm sorry, but I have to disagree with you."
B. "You're wrong and you should be ashamed of yourself."

{}
Who would say this? 'A' or 'B'?
"""

verbalizer4 = {"toxic": ["A"], "nontoxic": ["B"]}
prompt4 = """
Text: I understand where you're coming from, but I don't agree with your perspective.
Label: no
Text: You're a terrible person for thinking that way.
Label: yes
Text: {}
Label:
"""

verbalizer5 = {"toxic": [str(i) for i in range(5)], "nontoxic": [str(i) for i in range(6, 11)]}
prompt5 = """
How was this restaurant's dishes taste? Rate them with the score between 0 to 10.

{}
Answer with a score between 0 to 10.
"""

verbalizer6 = {"toxic": [str(i) for i in range(15)], "nontoxic": [str(i) for i in range(15, 100)]}
prompt6 =  """
If your server said this, how much would you like to tip them between 0 and 20%:

{}

Type a number between 0 and 20 that you will tip him.
"""

verbalizer7 = {"toxic": ["yes"], "nontoxic": ["no"]}
prompt7 = """
If it has uppercase words, it is toxic.
If it has the words 'fuck', 'asshole', 'dick', it is toxic.

Here is an example:

{}

Reply with yes or no. Is it toxic?
"""

verbalizer8 = {"toxic": ["yes"], "nontoxic": ["no"]}
prompt8 =  """
Text: I respect your opinion, but I have to disagree with it.
Label: no
Text: You're a horrible person for having that opinion.
Label: yes
Text: {}
Label:
"""

verbalizer9 = {"toxic": ["yes"], "nontoxic": ["no"]}
prompt9 = """
Justin is a polite guy who does not curse.
Evelyn is arude, obnoxious girl who says mean things.

Who said the following:
{}
If Justin, say no and if Evelyn, say yes.
"""

verbalizer10 = {"toxic": ["A"], "nontoxic": ["B"]}
prompt10 = """
A: "It's hilarious that people can't handle bullying."
B: "Cyberbullying is cruel. It's never okay."

Who said the following?
{}
A or B?
"""

In [15]:
# gdrive
# datapath = "drive/My Drive/SI 630/HW/Homework 4/"
# train_df = pd.read_csv(datapath + "jigsaw-train.csv")
# test_df = pd.read_csv(datapath + "jigsaw-test.csv")
# eval_df = pd.read_csv(datapath + "jigsaw-dev.csv")

train_df = pd.read_csv("jigsaw-train.csv")
test_df = pd.read_csv("jigsaw-test.csv")
eval_df = pd.read_csv("jigsaw-dev.csv")

In [16]:
test_df

,id,comment_text
0,d9836e25d089cab8,I suggest you add this to the LARPA wiki inste...
1,3fbed19498484f71,", 19 May 2008 (UTC) \n ::The AFD is truly sad ..."
2,be887f0617e43898,"===Train name, misnomer=== \n The problem is t..."
3,ddb1781c5174e079,March 2006
4,6f04966e1d4d2b61,unfair warnings as threats
...,...,...
32004,22abb35000de7828,== Motion Picture Association of America film ...
32005,834fd790ecbcf68f,""" \n\n ==WikiProject Pharmacology Update== \n ..."
32006,76417c0f552a71b2,That's nonsense. Most lists of Jews explicitly...
32007,6c7c337a97b1d905,==References== \n Removed Youtube link to V-En...


In [17]:
zeroshot_classifers = []
for i in tqdm(range(10)):
    sample = pd.concat([eval_df[eval_df["toxic"] == 1].sample(5), eval_df[eval_df["toxic"] == 0].sample(5)])
    zsclassifiers = []
    zs_f1_scores = []
    for i in range(1, 10):
        prompt_classifier = ZeroShotClassifier(eval("prompt" + str(i)), eval("verbalizer" + str(i)), tokenizer, model)
        zsclassifiers.append(prompt_classifier)
        _, score = prompt_classifier.evaluate(sample)
        zs_f1_scores.append(score)
    zeroshot_classifers.append(np.argmax(zs_f1_scores))
zeroshot_model_idx = max(set(zeroshot_classifers), key=zeroshot_classifers.count)
zeroshot_model = zsclassifiers[zeroshot_model_idx]
#sel_idx

  0%|          | 0/10 [00:00<?, ?it/s]
0it [00:00, ?it/s]
1it [00:02,  2.27s/it]
2it [00:02,  1.10s/it]
5it [00:02,  2.85it/s]
7it [00:02,  4.30it/s]
10it [00:03,  3.21it/s][A

0it [00:00, ?it/s]
1it [00:00,  4.28it/s]
2it [00:00,  3.32it/s]
3it [00:00,  3.86it/s]
4it [00:00,  4.56it/s]
6it [00:08,  1.98s/it]
10it [00:08,  1.20it/s][A

0it [00:00, ?it/s]
2it [00:00, 15.77it/s]
4it [00:00, 17.86it/s]
6it [00:00, 17.85it/s]
8it [00:00, 14.91it/s]
10it [00:00, 16.25it/s]

0it [00:00, ?it/s]
2it [00:00, 18.77it/s]
5it [00:00, 20.39it/s]
8it [00:00, 16.60it/s]
10it [00:00, 17.60it/s]

0it [00:00, ?it/s]
2it [00:00, 15.50it/s]
4it [00:00,  9.08it/s]
6it [00:00, 10.03it/s]
10it [00:00, 11.07it/s][A

0it [00:00, ?it/s]
2it [00:00, 15.41it/s]
4it [00:00, 15.91it/s]
6it [00:00, 14.39it/s]
8it [00:00, 11.87it/s]
10it [00:00, 13.04it/s]

0it [00:00, ?it/s]
2it [00:00, 16.21it/s]
4it [00:00, 17.08it/s]
6it [00:00, 16.96it/s]
8it [00:00, 14.20it/s]
10it [00:00, 15.41it/s]

0it [00:00, ?it/s]
2it [00

In [18]:
zs_f1_scores

[0.0,
 0.0,
 0.0,
 0.0,
 0.4444444444444445,
 0.6153846153846154,
 0.7142857142857143,
 0.0,
 0.0]

In [19]:
zs_prompt_classifier = ZeroShotClassifier(prompt7, verbalizer7, tokenizer, model)

In [20]:
zs_prompt_classifier

In [21]:
type(zs_prompt_classifier)

__main__.ZeroShotClassifier

In [22]:
zeroshot_model.template, zeroshot_model

('\nIf your server said this, how much would you like to tip them between 0 and 20%:\n\n{}\n\nType a number between 0 and 20 that you will tip him.\n',
 <__main__.ZeroShotClassifier at 0x14603d8a2bb0>)

## Problem 4. 
(10 points) For comparison with prompt-based learning models, train a regular classifier using Trainer and the MiniLM parameters on all the training data (very similar to what you did in Homework 3!). You should train your model for at least two epochs, but you’re not required to do any hyperparameter tuning (you just need a score). Predict the toxicity of the provided test data and calculate the F1.

## **Regular Classifier**

In [ ]:
import pandas as pd 
from collections import Counter, defaultdict
from tqdm import tqdm
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, mean_absolute_error

In [ ]:
torch.cuda.is_available()

True

In [ ]:
# Verify there's at least one GPU sitting around (or not if you want to try CPU!)
torch.cuda.device_count()

1

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding, AutoTokenizer, AutoModelForSequenceClassification
import torch
from datasets import Dataset, DatasetDict, load_dataset
import evaluate

In [ ]:
# Reformatting train and evaluation dataframes
# train_df = train_df.drop("id", axis=1)
# eval_df = eval_df.drop("id", axis=1)
train_df["toxic"] = train_df["toxic"].astype(int)
eval_df["toxic"] = eval_df["toxic"].astype(int)
# Testing data
# test_df = test_df.drop("id", axis=1)

In [ ]:
# train_dataset = Dataset.from_pandas(train_df)
# eval_dataset = Dataset.from_pandas(eval_df)

In [ ]:
##############################################################################################################

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
# tokenizer_miniLM = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')

# def preprocess_function(data):
#     return tokenizer_miniLM(data['comment_text'], padding=True, truncation=True, max_length=128)

# # tokenized_X_train = preprocess_function(list(train_df['comment_text']))
# # tokenized_X_eval = preprocess_function(list(eval_df['comment_text']))

# tokenized_X_train = train_dataset.map(preprocess_function, batched=True)
# tokenized_X_eval = eval_dataset.map(preprocess_function, batched=True)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer_miniLM)

In [ ]:
# # Define a Class to prepare the dataset in desired format to feed in Trainer class
# class DataProcess(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels = None):
#         self.encodings = encodings
#         self.labels = labels
  
#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         if self.labels:
#             item["toxic"] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.encodings["input_ids"])

In [ ]:
# train_dataset = DataProcess(tokenized_X_train, list(train_df['toxic']))
# eval_dataset = DataProcess(tokenized_X_eval, list(eval_df['toxic']))

In [ ]:
# from sklearn.metrics import mean_squared_error, f1_score

# import torch
# # from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
# # tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')
# # model = AutoModelForSequenceClassification.from_pretrained('microsoft/MiniLM-L12-H384-uncased', num_labels=2)

# def compute_metrics(pred):
#     scores = pred.label_ids
#     prediction = pred.predictions.squeeze()
#     return {"rmse": np.sqrt(mean_squared_error(scores, prediction))}

# # model_name = "microsoft/MiniLM-L12-H384-uncased"
# output_dir = '.'

# def model_init():
#     #tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')
#     miniLMmodel = AutoModelForSequenceClassification.from_pretrained('microsoft/MiniLM-L12-H384-uncased', num_labels=2)
#     model = miniLMmodel.to(device)

#     return model #AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
     
# training_args = TrainingArguments(
#     output_dir=output_dir,
#     evaluation_strategy='epoch',     # evaluate after each epoch
#     save_strategy='epoch',           # save after each epoch
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=2,
#     weight_decay=0.01,
#     do_eval=True,
#     seed=1234,
#     push_to_hub=False,
#     logging_dir=output_dir + 'logs/',
#     logging_steps=10,
#     load_best_model_at_end=True,
#     #metric_for_best_model='accuracy',
#     metric_for_best_model="f1",
# )

In [ ]:
##############################################################################################################

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')

def preprocess_function(data):
    return tokenizer(data, padding=True, truncation=True, max_length=512)

tokenized_X_train = preprocess_function(list(train_df['comment_text']))
tokenized_X_eval = preprocess_function(list(eval_df['comment_text']))

In [ ]:
# Define a Class to prepare the dataset in desired format to feed in Trainer class
class DataProcess(torch.utils.data.Dataset):
    def __init__(self, encodings, labels = None):
        self.encodings = encodings
        self.labels = labels
  
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
train_dataset = DataProcess(tokenized_X_train, list(train_df['toxic']))
eval_dataset = DataProcess(tokenized_X_eval, list(eval_df['toxic']))

In [ ]:
device = torch.device("cuda")

In [ ]:
from sklearn.metrics import mean_squared_error

import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
# tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')
# model = AutoModelForSequenceClassification.from_pretrained('microsoft/MiniLM-L12-H384-uncased', num_labels=2)

def compute_metrics(pred):
    scores = pred.label_ids
    prediction = pred.predictions.squeeze()
    return {"rmse": np.sqrt(mean_squared_error(scores, prediction))}

# model_name = "microsoft/MiniLM-L12-H384-uncased"
output_dir = '.'

def model_init():
    #tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')
    miniLMmodel = AutoModelForSequenceClassification.from_pretrained('microsoft/MiniLM-L12-H384-uncased', num_labels=2)
    model = miniLMmodel.to(device)

    return model
     
training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy='epoch',     # evaluate after each epoch
    save_strategy='epoch',           # save after each epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,

    num_train_epochs=3,
    weight_decay=0.01,
    do_eval=True,
    push_to_hub=False,
    logging_dir=output_dir + 'logs/',
    logging_steps=10,
    load_best_model_at_end=True,
    #metric_for_best_model='accuracy',
    metric_for_best_model="rmse",
)

In [ ]:
trainer = Trainer(
    model_init=model_init,    
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,    
    compute_metrics = compute_metrics, # compute_metrics=lambda pred: {'accuracy': (pred.predictions.argmax(-1) == pred.label_ids).mean()},
    tokenizer=tokenizer,
)

loading configuration file config.json from cache at /home/clairewo/.cache/huggingface/hub/models--microsoft--MiniLM-L12-H384-uncased/snapshots/44acabbec0ef496f6dbc93adadea57f376b7c0ec/config.json
Model config BertConfig {
  "_name_or_path": "microsoft/MiniLM-L12-H384-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/clairewo/.cache/huggingface/hub/models--microsoft--MiniLM-L12-H384-uncased/snapshots/44acabbec0ef496f6dbc93adadea57f376b7c0ec/pytorch_model.bin
All mo

In [ ]:
#!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
trainer.train()

loading configuration file config.json from cache at /home/clairewo/.cache/huggingface/hub/models--microsoft--MiniLM-L12-H384-uncased/snapshots/44acabbec0ef496f6dbc93adadea57f376b7c0ec/config.json
Model config BertConfig {
  "_name_or_path": "microsoft/MiniLM-L12-H384-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/clairewo/.cache/huggingface/hub/models--microsoft--MiniLM-L12-H384-uncased/snapshots/44acabbec0ef496f6dbc93adadea57f376b7c0ec/pytorch_model.bin
All mo

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: clairewo (clairewoo). Use `wandb login --relogin` to force relogin


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [ ]:
# pip install wandb

In [ ]:
# import wandb
# import random

# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="SI630-HW4-miniLM-regular-classifier",
    
#     # track hyperparameters and run metadata
#     config={
#     "learning_rate": 0.02,
#     "architecture": "CNN",
#     "dataset": "CIFAR-100",
#     "epochs": 10,
#     }
# )

# # simulate training
# epochs = 10
# offset = random.random() / 5
# for epoch in range(2, epochs):
#     acc = 1 - 2 ** -epoch - random.random() / epoch - offset
#     loss = 2 ** -epoch + random.random() / epoch + offset
    
#     # log metrics to wandb
#     wandb.log({"acc": acc, "loss": loss})
    
# # [optional] finish the wandb run, necessary in notebooks
# wandb.finish()

In [ ]:
# wandb login

## **Comparison and Evaluation**

## Problem 5. 
(20 points) Using your patterns and verbalizers, train separate prompt-based learning models on 10, 50, 100, and 500 instances of data. Your data should be randomly sampled from the training data but be sure to have examples of each class. You are free to choose which instances you use and what distribution of toxic/non-toxic labels are in your training data (provided you have at least one example of each). For each model, predict the scores for the provided development data and calculate the Binary F1 (toxic is the positive class).

In [66]:
train_df[train_df["toxic"]== 1]

,comment_text,toxic
6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
12,Hey... what is it..\n@ | talk .\nWhat is it......,1
16,"Bye! \n\nDon't look, come or think of comming ...",1
42,You are gay or antisemmitian? \n\nArchangel WH...,1
43,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1
...,...,...
159494,"""\n\n our previous conversation \n\nyou fuckin...",1
159514,YOU ARE A MISCHIEVIOUS PUBIC HAIR,1
159541,Your absurd edits \n\nYour absurd edits on gre...,1
159546,"""\n\nHey listen don't you ever!!!! Delete my e...",1


In [68]:
df_10_1 = train_df[train_df["toxic"]== 1].sample(n=5) # toxic
df_10_2 = train_df[train_df["toxic"]== 0].sample(n=5) # nontoxic

df_10 = pd.concat([df_10_1, df_10_2])
df_10

,comment_text,toxic
77989,"yes, and lets also delete or change any articl...",1
154931,Do you want me to ban you? I am prejudice agai...,1
142348,"FUCK YOU!!!!=\n FUCK YOU, YOU STUPID BASTARD...",1
119011,Rats at the Wikipedia web site \n\nRats are ar...,1
33268,He hadn't been in the USA since Bill Clinton w...,1
142266,The thing to remember is I and a lot of others...,0
119065,I suggest you pick out the specific claims in ...,0
58736,January 2009 \n\nFine then. I'll stop.,0
40156,"Despite my last comment, you kept on edit-warr...",0
48024,There are several derogatory remarks made here...,0


In [70]:
df_50_1 = train_df[train_df["toxic"]== 1].sample(n=25) # toxic
df_50_2 = train_df[train_df["toxic"]== 0].sample(n=25) # nontoxic

df_50 = pd.concat([df_50_1, df_50_2])
df_50.head()

,comment_text,toxic
46361,What the fuck are you talking about?,1
38512,Let's get to fuckin big daddy ==\nMan I love s...,1
109209,"""\n\nWhat right do you have to block the White...",1
44369,fuck u i will do as i please,1
88341,Remove \n\nNeilN needs to extract his head fro...,1


In [71]:
df_500_1 = train_df[train_df["toxic"]== 1].sample(n=250) # toxic
df_500_2 = train_df[train_df["toxic"]== 0].sample(n=250) # nontoxic

df_500 = pd.concat([df_500_1, df_500_2])
df_500.head()

,comment_text,toxic
442,Hi \n\nIm a fucking bitch.\n\n50.180.208.181,1
131453,BEST BEHAVIOUR IS A SINGLE! \n\nhttp://www.ndu...,1
36587,be a toilet. so everyone can piss her off. hey...,1
96561,is all pretty gay if u ask me...bunch or fucki...,1
88927,Hey you! \n\nI'll strike you down without a se...,1


In [72]:
len(df_500)

500

## Problem 6. 
(10 points) Let’s compare our two prompt-based models (the few-shot and zero-shot) and our regular all-data MiniLM model. Plot the score for each prompt-based learning model and your full-data MiniLM model using Seaborn. Use the training data size as the x-axis and Binary F1 as the y-axis. For the zero-shot models, you can draw these as different horizontal lines or points (since they don’t have training data amounts) If you are feeling curious, feel free to train regular models on different sizes/distributions of data and include those too. Write your guess on how many instances you think you need to train a prompt-based learning model that will reach the performance of a MiniLM model trained on all the data.

## Problem 7. 
(10 points) How good can we get our zero-shot model? Pick your favorite zero-shot prompt and predict the labels for the provided test data. Upload these predictions to the Kaggle competition posted about on Piazza. Scores within some reasonable range (e.g., not zero or near-zero) will receive full credit, though aim high.

In [23]:
model_pred = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
tokenizer_pred = AutoTokenizer.from_pretrained("google/flan-t5-small")

In [24]:
zs_prompt_classifier = ZeroShotClassifier(prompt6, verbalizer6, tokenizer_pred, model_pred)

In [25]:
zs_prompt_classifier

In [26]:
# # tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')

# def preprocess_function_pred(data):
#     return tokenizer_pred(data, padding=True, truncation=True, max_length=512)

# tokenized_X_test = preprocess_function_pred(list(test_df['comment_text']))

In [27]:
# test_dataset = DataProcess(tokenized_X_test) 
# test_dataset

In [28]:
# zeroshot_model.evaluate(test_df, eval=False)

In [29]:
# test_df = pd.read_csv("jigsaw-test.csv")

In [30]:
test_df

,id,comment_text
0,d9836e25d089cab8,I suggest you add this to the LARPA wiki inste...
1,3fbed19498484f71,", 19 May 2008 (UTC) \n ::The AFD is truly sad ..."
2,be887f0617e43898,"===Train name, misnomer=== \n The problem is t..."
3,ddb1781c5174e079,March 2006
4,6f04966e1d4d2b61,unfair warnings as threats
...,...,...
32004,22abb35000de7828,== Motion Picture Association of America film ...
32005,834fd790ecbcf68f,""" \n\n ==WikiProject Pharmacology Update== \n ..."
32006,76417c0f552a71b2,That's nonsense. Most lists of Jews explicitly...
32007,6c7c337a97b1d905,==References== \n Removed Youtube link to V-En...


In [31]:
zs_preds = zeroshot_model.evaluate(test_df, eval=False)

32009it [2:11:21,  4.06it/s]


In [32]:
zeroshot_predictions = np.array(zs_preds[0])

In [33]:
len(zs_preds[0])

32009

In [34]:
test_df["toxic"] = zeroshot_predictions

In [35]:
test_df.to_csv("sample_jigsaw-submission_clairewo.csv", index=False)

In [36]:
new_test_df = test_df.drop(columns = ["comment_text"]).reset_index(drop=True)

In [37]:
new_test_df.to_csv("clairewo_jigsaw-submission.csv", index=False)

In [38]:
check = pd.read_csv("clairewo_jigsaw-submission.csv")

In [39]:
check

,id,toxic
0,d9836e25d089cab8,0
1,3fbed19498484f71,1
2,be887f0617e43898,1
3,ddb1781c5174e079,1
4,6f04966e1d4d2b61,1
...,...,...
32004,22abb35000de7828,1
32005,834fd790ecbcf68f,1
32006,76417c0f552a71b2,1
32007,6c7c337a97b1d905,0
